weaviate 벡터스토어 구성


In [1]:
from langchain_community.embeddings import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    base_url = 'http://192.168.1.203:11434',
    model="bge-m3:latest",
    # model="chatfire/bge-m3:q8_0" # BGE-M3
)
import weaviate
from weaviate.classes.config import Property, DataType, Configure

C:\Users\kevin\AppData\Local\Temp\ipykernel_25616\3807986265.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(
d:\conda_env\langmanus\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.0 at grpc_health/v1/health.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\conda_env\langmanus\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.0 at v1/aggre

In [2]:
# client = weaviate.connect_to_local()
# Weaviate가 http://localhost:8081 에서 실행 중인 경우
client = weaviate.connect_to_local(host="192.168.1.203", port=8585)

In [12]:
client.collections.delete("m_asset_sample_query_2_hint")

In [13]:
# 스키마 정의
properties = [
    Property(name="query", data_type=DataType.TEXT),
    Property(name="explanation", data_type=DataType.TEXT),
]
collection_name = "m_asset_sample_query_2_hint"
if collection_name in client.collections.list_all():
    client.collections.delete(name=collection_name)
# 컬렉션 생성
client.collections.create(
    name="m_asset_sample_query_2_hint",
    properties=properties,
    vectorizer_config=None,  # 벡터화를 직접 처리할 예정이므로 'None'으로 설정
    # vectorizer_config=Configure.Vectorizer.text2vec_ollama(   # hybrid 에서 필요
    #         model="bge-m3:latest" # 사용하는 Ollama 임베딩 모델 이름 (예: nomic-embed-text)
    #     ),
    # Hybrid 검색을 위한 인덱스 설정
    inverted_index_config=Configure.inverted_index(
        bm25_b=0.75,
        bm25_k1=1.2,
        # cleanup_interval_seconds=60,
        index_null_state=False,
        index_property_length=False,
        index_timestamps=False,
        # stopwords_preset="en"
    )
)

# news_collection = client.collections.get("m_asset")
news_collection = client.collections.get("m_asset_sample_query_2_hint")


In [14]:
sample_query = """
select
	s.kor_name_small as 종목명,
	e.high_52w_price as 연중최고가,
	e.foreigner_limit_ratio as 외국인보유율, 
	e.trading_volume as 거래량,
	s.short_sell_volume as 공매도,
	(e.close_price - e.prev_close_price) / e.prev_close_price * 100 as 상승율,
	s.prev_year_per AS PER, 
	s.dividend_yield as 배당수익률
from
	m_asset.exchange_kosdaq_stock_master_01 e
join m_asset.exchange_kosdaq_stock_master s on
	e.stock_code = s.stock_code AND e.data_date = s.data_date;
"""
explanation = "연중 최고가, 외국인보유율, 거래량, 공매도, 상승률, PER, 배당수익률"

explanation_embedding = ollama_embeddings.embed_query(explanation)
news_collection.data.insert(
    properties={"query": sample_query,"explanation": explanation},
    vector=explanation_embedding
)


UUID('0cea14e7-44bd-4a3b-9763-8d774ebb26c8')

In [15]:
sample_query = """
select
	s.kor_name_small as 종목명,
	e.accumulated_trade_amount as 누적거래대금
from
	m_asset.kosdaq_trade_execution e
join m_asset.exchange_kosdaq_stock_master s on
	e.stock_code = s.stock_code AND e.data_date = s.data_date;
"""
explanation = "누적거래대금"

explanation_embedding = ollama_embeddings.embed_query(explanation)
news_collection.data.insert(
    properties={"query": sample_query,"explanation": explanation},
    vector=explanation_embedding
)


UUID('525478e7-99c0-428b-a0b0-53f2f4caa98a')